In [121]:
import pandas as pd
import numpy as np
import math as m

In [122]:
df = pd.read_csv('data/data.csv')

In [123]:
# Создаем столбцы с пролагарифмированными данными
df['X'] = np.log(df['x'])
df['Y'] = np.log(df['y'])

In [124]:
df['xy'] = df['x'] * df['y']
df['x2'] = df['x'] ** 2
df['y2'] = df['y'] ** 2

In [125]:
df

,x,y,X,Y,xy,x2,y2
0,78,133,4.356709,4.890349,10374,6084,17689
1,82,148,4.406719,4.997212,12136,6724,21904
2,87,134,4.465908,4.897840,11658,7569,17956
3,79,154,4.369448,5.036953,12166,6241,23716
4,89,162,4.488636,5.087596,14418,7921,26244
5,106,195,4.663439,5.273000,20670,11236,38025
6,67,139,4.204693,4.934474,9313,4489,19321
7,88,158,4.477337,5.062595,13904,7744,24964
8,73,152,4.290459,5.023881,11096,5329,23104
9,87,162,4.465908,5.087596,14094,7569,26244


In [127]:
b = (df['xy'].mean() - df['x'].mean() * df['y'].mean()) / (df['x'].std() ** 2)
a = df['y'].mean() - b * df['x'].mean()

In [128]:
print(f'y = {a} + {b} * x')

y = 83.5409447664387 + 0.8437280066238907 * x


В среднем, при увеличении среднедушевого прожиточного минимума на 1 рубль, среднедневная ЗП увеличивается в среднем на 0.84 рубля

In [129]:
def f(x, a, b):
    return a + b * x

In [130]:
df['y^'] = f(df['x'], a, b) 

In [131]:
df['y-y^'] = df['y'] - df['y^']

In [132]:
df[['y^', 'y-y^']]

,y^,y-y^
0,149.351729,-16.351729
1,152.726641,-4.726641
2,156.945281,-22.945281
3,150.195457,3.804543
4,158.632737,3.367263
5,172.976113,22.023887
6,140.070721,-1.070721
7,157.789009,0.210991
8,145.133089,6.866911
9,156.945281,5.054719


In [133]:
df['Ai'] = abs( (df['y-y^']) * 100 / (df['y']) )

In [134]:
df[['Ai']]

,Ai
0,12.294533
1,3.193677
2,17.123344
3,2.470482
4,2.078557
5,11.294301
6,0.770303
7,0.133538
8,4.517704
9,3.120197


In [135]:
A_mean = df['Ai'].mean()
print(f'Средняя ошибка = {A_mean} %.')

Средняя ошибка = 5.708462836670443 %.


В среднем, расчетные значения отклоняются от фактических на 8.1 %.

In [136]:
y_mean = df['y'].mean()
if df['x'].std() > df['y'].std():
    r = b * df['y'].std() / df['x'].std()
else: 
    r = b * df['x'].std() / df['y'].std()
print(f'Коэффициент парной корреляции = {r}')

Коэффициент парной корреляции = 0.6609397792839882


Корреляция умереная.

## Проверка значимости коэффициентов регрессии и корреляции

**$H_0$**: Выявленная зависимость обусловленна влиянием случайности

In [137]:
mr = np.sqrt((1 - r ** 2) / (len(df['x']) - 2))
ma = np.sqrt( (sum(df['y-y^'] ** 2) / (len(df['x']) - 2)) * ((sum(df['x'] ** 2)) / (len(df['x']) * sum((df['x'] - df['x'].mean()) ** 2))) )
mb = np.sqrt((sum(df['y-y^'] ** 2)) / (len(df['x'] - 2)) / (sum((df['x'] - df['x'].mean()) ** 2)))

In [138]:
print(*[ta := a / ma,  tb := b / mb, tr := r / mr])

3.4375566565821605 3.2919232472779645 2.785136774159887


In [139]:
print(t := 2.22813885196)

2.22813885196


$t_a$ = 3.4375566565821605

$t_b$ = 3.2919232472779645

$t_r$ = 2.785136774159887

$t_{таб}$ = 2.22813885196

In [140]:
x_p = 1.07 * df['x'].mean()
y_p = f(x_p, a, b)
print(f'Прогнозное x = {x_p}; прогнозное значение y = {y_p}')

Прогнозное x = 91.57416666666667; прогнозное значение y = 160.80463386634932


Так как:

$t_a$ > $t_{таб}$

$t_b$ > $t_{таб}$

$t_r$ > $t_{таб}$

$\implies$ имеем достаточно оснований отклонить гипотезу $H_0$

$\implies$ Выявленная зависимость значима.

In [141]:
my = np.sqrt((sum(df['y-y^'] ** 2)) / (len(df['x'])) * (1 + 1 / len(df['x']) + 
                                                        ((x_p - df['x'].mean()) ** 2) / (sum((df['x'] - df['x'].mean()) ** 2))))

In [142]:
delta = my * t

In [143]:
my

12.066783245225924

In [144]:
print(f'Доверительный интервал ({y_p - delta}; {y_p + delta})')

Доверительный интервал (133.91816529948147; 187.69110243321717)


Доверительный интервал прогноза

$\overset{\wedge}{y_p}  - \Delta_{\overset{\wedge}{y_p}} < y < \overset{\wedge}{y_p}  + \Delta_{\overset{\wedge}{y_p}}$

133.91816529948147 $< y <$ 187.69110243321717